Арцишевский Антон, 183 группа, дз 2.1

In [1]:
from pymystem3 import Mystem
import json
%load_ext pycodestyle_magic
%pycodestyle_on

### **Задание: Обработать книгу с помощью mystem:**
##### **Просто читаю книгу, парсю майстемом и записываю как json в отдельный файл**

In [2]:
import time
start = time.time()
m = Mystem()
with open('feelpower_Asimov.txt', encoding='utf-8') as text:
    text = text.read()
lemmas = m.analyze(text)
with open('mystem_Asimov.json', 'w', encoding='utf-8') as f:
    json_mystem_Asimov = json.dump(lemmas, f, ensure_ascii=False)
end = time.time()
print('Затраченное время:', end - start)

Затраченное время: 154.211284160614


### **Задание: Обработать книгу через pymorphy**
##### **Чищу текст, читаю разбор каждого слова, из которого беру нужные показатели. Запихиваю это в виде словаря в заранее созданный список и записываю его в качестве json строки в файл**

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

In [4]:
start = time.time()
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
morph = MorphAnalyzer()
gram_razbor = []
for word in words:
    morph_asimov = morph.parse(word)
    wordform = morph_asimov[0].word
    lexemma = morph_asimov[0].normal_form
    tags = morph_asimov[0].tag.POS
    simple_list = [{'СЛОВО': wordform, 'ЛЕКСЕМА': lexemma, 'ЧАСТЬ РЕЧИ': tags}]
    gram_razbor.extend(simple_list)
with open('pymorphy_Asimov.json', 'w', encoding='utf-8') as f:
    json_pymorpy_Asimov = json.dump(gram_razbor, f, ensure_ascii=False)
end = time.time()
print('Затраченное время:', end - start)

Затраченное время: 0.9683258533477783


### **Задание: Какую долю слов составляет каждая часть речи?**
##### **Беру разбор слова. Когда программа дойдёд до части речи, то я беру значение части речи и добавляю его в заранее созданный список. Получается список с перечеслением всех частей речи всех слов в тексте. Я считаю их с помощью Counter. Затем я считаю частотность для каждой части речи**

In [5]:
from collections import Counter
word_classes = []
freq = []
for i in gram_razbor:
    for pare in i.items():
        if 'ЧАСТЬ РЕЧИ' in pare:
            word_class = pare[1]
            word_classes.append(word_class)
all_words = len(words)
c = Counter(word_classes)
values = c.items()
print('Частота частей речи:')
for x in values:
    freq_class = x[1] / all_words
    print(x[0], '-', freq_class)

Частота частей речи:
NOUN - 0.25626423690205014
VERB - 0.14502657555049356
INFN - 0.0425208807896735
PREP - 0.0914958238420653
ADJF - 0.10516324981017464
PRTF - 0.008352315869400152
NPRO - 0.08466211085801063
CONJ - 0.10364464692482915
ADVB - 0.05353075170842825
ADJS - 0.011009870918754746
PRCL - 0.06340167046317388
None - 0.009870918754745633
NUMR - 0.004555808656036446
COMP - 0.007593014426727411
PRED - 0.006454062262718299
GRND - 0.0026575550493545936
INTJ - 0.002277904328018223
PRTS - 0.0015186028853454822


### **Задание: Найдите топ-20 (по частотности) глаголов и наречий**
##### **Перебираю разборы. Если в значениях разбора встречается глагол, то я создаю переменную лексемы глагола и пихунькаю её в заранее созданный словарь (либо обновляю его, если переменная уже лежит в словаре). Потом я сортирую топ-20 и красивенько его принчу. Повторяю операцию для наречий**

In [6]:
import operator
verbs = {}
for i in gram_razbor:
    if 'VERB' in i.values():
        lexemma = i['ЛЕКСЕМА']
        if lexemma in verbs:
            verbs[lexemma] += 1
        else:
            verbs[lexemma] = 1
sorted_verbs = sorted(verbs.items(), key=operator.itemgetter(1), reverse=True)
print('Топ-20 глаголов и их кол-во вхождений:')
for x, y in sorted_verbs[0:20]:
    print(x, '-', y)
print('')

advbs = {}
for i in gram_razbor:
    if 'ADVB' in i.values():
        lexemma = i['ЛЕКСЕМА']
        if lexemma in advbs:
            advbs[lexemma] += 1
        else:
            advbs[lexemma] = 1
sorted_advbs = sorted(advbs.items(), key=operator.itemgetter(1), reverse=True)
print('Топ-20 наречий и их кол-во вхождений:')
for x, y in sorted_advbs[0:20]:
    print(x, '-', y)

Топ-20 глаголов и их кол-во вхождений:
быть - 45
мочь - 28
сказать - 16
говорить - 7
возразить - 6
видеть - 6
произнести - 5
хотеть - 5
удаться - 5
писать - 5
продолжать - 5
достать - 4
делать - 4
думать - 4
получаться - 4
подумать - 4
суметь - 4
увидеть - 3
спросить - 3
перемножить - 3

Топ-20 наречий и их кол-во вхождений:
сейчас - 7
потом - 7
всегда - 6
теперь - 6
потому - 4
пока - 4
очень - 4
тоже - 3
хорошо - 3
снова - 3
всего - 2
несколько - 2
совсем - 2
там - 2
слегка - 2
нетерпеливо - 2
много - 2
почему - 2
вдруг - 2
трудно - 2


### **Задание: топ-25 биграмм и триграмм**
##### **Удаляю пунктуацию, ищу биграммы по распличенному тексту, беру топ-25 и красиво печатаю. Повторяю операцию с триграммами**

In [7]:
import string
b_result = ""
for c in text:
    if c not in string.punctuation:
        b_result += c
bigrm = list(nltk.bigrams(b_result.split()))
c = Counter(bigrm)
z = c.most_common(25)
print('Самые частотные биграммы:')
for key, value in z:
    print(key[0], '+', key[1], '-', value)

print('')

t_result = ""
for c in text:
    if c not in string.punctuation:
        t_result += c
bigrm = list(nltk.trigrams(t_result.split()))
c = Counter(bigrm)
z = c.most_common(25)
print('Самые частотные триграммы:')
for key, value in z:
    print(key[0], '+', key[1], '-', value)

Самые частотные биграммы:
Я + не - 7
и + не - 5
Генерал + Уэйдер - 4
умножить + на - 4
так + что - 4
техник + Ауб - 4
Он + был - 3
возразил + Шуман - 3
на + бумаге - 3
как + вы - 3
на + 3 - 3
Это + значит - 3
можно + будет - 3
без + счетчика - 3
что + это - 3
и + в - 3
Я + могу - 3
говорит + что - 3
может + воспроизвести - 3
я + вижу - 3
Программист + Шуман - 3
вычислительных + машин - 3
конце + концов - 3
больше + не - 3
счетных + машин - 2

Самые частотные триграммы:
достал + свой - 2
7 + умножить - 2
умножить + на - 2
так + что - 2
Шуман + говорит - 2
теперь + я - 2
В + конце - 2
но + техник - 2
Айзек + Азимов - 1
Азимов + Чувство - 1
Чувство + силы - 1
силы + Джехан - 1
Джехан + Шуман - 1
Шуман + привык - 1
привык + иметь - 1
иметь + дело - 1
дело + с - 1
с + высокопоставленными - 1
высокопоставленными + людьми - 1
людьми + руководящими - 1
руководящими + раздираемой - 1
раздираемой + распрями - 1
распрями + планетой - 1
планетой + Он - 1
Он + был - 1
